<p style="text-align: center;font-size: 40pt">Testing code 3D</p>

In [1]:
%matplotlib widget
#%matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import proj3d

import numpy as np

import ipywidgets as widgets

%run ./scripts/helper_func.py
path = "{0}/common/scripts/style.py".format(get_root_path())
%run $path

# Overview 

Requirements
- [Rigid transformation in 2D](../transformations_2d/3-lesson_rigid.ipynb)

Objectives of this lesson:

- review 2D transformation functions
- introduce part of the notations used later for 3D rigid transformation
- train on easy to view problems before moving to 3D


Hidden custom latex commands here $ \curvearrowright$

----
[comment]: <> (General commands)
$\newcommand{\textcomma}{\quad\text{,}}$
$\newcommand{\textdot}{\quad\text{.}}$
$\newcommand{\vec}[1]{\overrightarrow{#1}}$
$\newcommand{\mat}[1]{\mathbf{#1}}$
$\newcommand{\frame}[1]{\mathcal{#1}}$
$\newcommand{\point}[2][]{{}^{#1}\mathbf{#2}}$
$\newcommand{\pointsym}[2][]{{}^{#1}\boldsymbol{#2}}$
$\newcommand{\matsym}[1]{\boldsymbol{#1}}$
$\newcommand{\real}{\mathbb{R}}$
$\newcommand{\bmat}[1]{\begin{bmatrix}#1\end{bmatrix}}$
$\newcommand{\F}[2][]{{}_{#2}^{#1}\mathscr{F}}$
$\newcommand{\Fmat}[2][]{{}_{#2}^{#1}\mat{F}}$
$\newcommand{\origin}[2][]{{}_{#2}^{#1}\mat{o}}$
$\newcommand{\T}[2][]{{}_{#2}^{#1}\mat{T}}$
$\newcommand{\t}[2][]{{}_{#2}^{#1}\mat{t}}$
$\newcommand{\R}[2][]{{}_{#2}^{#1}\mat{R}}$
$\newcommand{\f}{\vec{\mathscr{f}}}}$
$\newcommand{\ax}[2][]{{}_{#2}^{#1}\vec{\mathscr{x}}}$
$\newcommand{\ay}[2][]{{}_{#2}^{#1}\vec{\mathscr{y}}}$
$\newcommand{\az}[2][]{{}_{#2}^{#1}\vec{\mathscr{z}}}$
$\newcommand{\aw}[2][]{{}_{#2}^{#1}\vec{\mathscr{w}}}$
$\newcommand{\axi}{\mathscr{x}}$
$\newcommand{\ayi}{\mathscr{y}}$
$\newcommand{\azi}{\mathscr{z}}$
$\newcommand{\awi}{\mathscr{w}}$
$\newcommand{\pointx}[2][]{{}^{#1}{#2}_{\axi}}$
$\newcommand{\pointy}[2][]{{}^{#1}{#2}_{\ayi}}$
$\newcommand{\pointz}[2][]{{}^{#1}{#2}_{\azi}}$
$\newcommand{\SO}[1]{\mathrm{SO}(#1)}$
----

In [2]:
%matplotlib widget
#%matplotlib inline

from matplotlib.patches import FancyArrowPatch
from matplotlib.text import Text

def draw_3d_vector(ax, head, text="", origin=[0,0,0], text_offset=[0,0,0]):
    text_global = origin + head + text_offset
    quiver_handle = ax.quiver(origin[0], origin[1], origin[2], 
               head[0], head[1], head[2], length=1., normalize=True, linewidths=0.5)
    
    x2, y2, _ = proj3d.proj_transform(text_global[0], text_global[1], text_global[2], ax.get_proj())
    text_handle = ax.text2D(x2, y2, text, size=20)
    return (quiver_handle, text_handle)

def rotation_matrix_z(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta),  np.cos(theta), 0],
                     [0,0,0]])

#TODO: do the same with text...
class Arrow3D(FancyArrowPatch):
    def __init__(self, head, origin=[0,0,0], *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        xs = [origin[0], head[0]]
        ys = [origin[1], head[1]]
        zs = [origin[2], head[2]]
        self._verts3d = xs, ys, zs
        self.init_style = self.get_arrowstyle()

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        print(renderer.M)
        FancyArrowPatch.set_positions(self,(xs[0],ys[0]),(xs[1],ys[1]))
        dx = np.abs(xs[0] - xs[1])
        dy = np.abs(ys[0] - ys[1])
        thresh = 0.04
        if dx < thresh and dy < thresh:
            self.set_arrowstyle("->",head_length=dx*10, head_width=dx*5.)
        else:
            self.set_arrowstyle(self.init_style)
        FancyArrowPatch.draw(self, renderer)
        
    def set_positions(self, head, origin=[0,0,0]):
        xs = [origin[0], head[0]]
        ys = [origin[1], head[1]]
        zs = [origin[2], head[2]]
        self._verts3d = xs, ys, zs

        
class Annotation3D(Annotation):
    '''Annotate the point xyz with text s'''

    def __init__(self, s, xyz, text_offset=[0,0,0],*args, **kwargs):
        Annotation.__init__(self,s, xy=(0,0), *args, **kwargs)
        self._verts3d = xyz + text_offset
        self._offset = text_offset

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        Annotation.set_position(self, (xs,ys))
        Annotation.draw(self, renderer)
        
    def set_position(self, xyz):
        self._verts3d = xyz + self._offset
        
test = np.array([1,0,0])
a = np.array([1,0,0])

R = rotation_matrix_z(theta=np.pi/2)
b = R @ a
c = np.cross(a,b)

#plt.close(fig)
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111,projection="3d")

lines  = ax.plot3D([],[],[])

print(lines)

arr = Arrow3D(b, mutation_scale=20, 
                lw=3, arrowstyle="->", color="r")
ax.add_artist(arr)

ann = Annotation3D(r"$\mathbf{b}$", b, text_offset=[0,0,-0.2],size=20)
ax.add_artist(ann)

ax.set_xlim(-1, 1); ax.set_ylim(-1, 1); ax.set_zlim(-1, 1)
ax.set_xlabel("X")
#text_handle = ax.text2D(0, 0, "b", size=20)


#text=ax.text2D(0,0, "", va="bottom", ha="left")
#fig.canvas.draw()

def update(angle=0.0):
    global b
    global quiver_handle
    R = rotation_matrix_z(angle)
    b = R @ a
    c = np.cross(a,b)
    
    arr.set_positions(b)
    ann.set_position(b)
    
    #x2, y2, _ = proj3d.proj_transform(b[0], b[1], b[2], ax.get_proj())
    #text_handle.set_position((x2,y2))
    
    fig.canvas.draw() #TODO find refreshing bug here

#def on_motion(event):
#    x2, y2, _ = proj3d.proj_transform(b[0], b[1], b[2], ax.get_proj())
#    text_handle.set_position((x2,y2))
    #tx = 'button=%d, x=%d, y=%d, xdata=%f, ydata=%f' % (event.button, event.x, event.y, event.xdata, event.ydata)
    #tx = '%f, %f' % (x2,y2)
    #text.set_text(tx)

#cid = fig.canvas.mpl_connect('motion_notify_event', on_motion)
#cid = fig.canvas.mpl_connect('draw_event', on_motion)
widgets.interact(update, angle = (0, np.pi, 0.1), continuous_update=True);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='angle', max=3.141592653589793), Output()), _dom_clas…